In [2]:
# pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 7.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.4 MB/s eta 0:00:0000:0100:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached sympy-1.11.1-py3-none-any.whl (6.5 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import torch.nn as nn
class DecisionTransformer(nn.Module):
    def __init__(self, state_dim, action_dim, reward_dim, embedding_dim, nhead, num_layers):
        super(DecisionTransformer, self).__init__() 
        self.state_embedding = nn.Linear(state_dim, embedding_dim) 
        self.action_embedding = nn.Linear(action_dim, embedding_dim) 
        self.reward_embedding = nn.Linear(reward_dim, embedding_dim)
        
        self.transformer = nn.Transformer(embedding_dim, nhead, num_layers) 
        self.output_layer = nn.Linear(embedding_dim, action_dim)
        
    def forward(self, states, actions, rewards, mask=None): 
        state_embeds = self.state_embedding(states) 
        action_embeds = self.action_embedding(actions) 
        reward_embeds = self.reward_embedding(rewards)
        input_embeds = torch.cat((state_embeds, action_embeds, reward_embeds), dim=1) 
        # tgt = torch.rand((20, 32, 512))
        # TODO: what is tgt?
        transformer_output = self.transformer(input_embeds.transpose(0, 1), src_key_padding_mask=mask, tgt=input_embeds.transpose(0, 1)) 
        action_logits = self.output_layer(transformer_output[-1])
        return action_logits

state_dim = 10 
action_dim = 5 
reward_dim = 1 
embedding_dim = 64 
nhead = 4 
num_layers = 2

model = DecisionTransformer(state_dim, action_dim, reward_dim, embedding_dim, nhead, num_layers)

# Load dataset
states = torch.randn(32, 10, state_dim)
actions = torch.randint(0, action_dim, (32, 10, 1))
rewards = torch.randn(32, 10, 1)

# Transfer to one-hot
actions_one_hot = torch.zeros(32, 10, action_dim) 
actions_one_hot.scatter_(2, actions, 1)

# Got outputs of the model
action_logits = model(states, actions_one_hot, rewards)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Training (one batch)
optimizer.zero_grad()
loss = loss_fn(action_logits, actions.squeeze(-1)[:, -1]) 
loss.backward()
optimizer.step()